Inspect genus-to-genus taxonomic mapping based on MGBC toolkit table:


In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [6]:
mgbc_clostest_taxa_table_path = '/home/noa/lab_code/MGBC-Toolkit/data/closest_tax.tsv'
mgbc_clostest_taxa_table = pd.read_csv(mgbc_clostest_taxa_table_path, sep='\t', header=None, names = ['method', 'reference_genome', 'query_genome', 'distance', 'reference_taxonomy', 'query_taxonomy'])

In [7]:
mgbc_clostest_taxa_table.head()

,method,reference_genome,query_genome,distance,reference_taxonomy,query_taxonomy
0,taxonomy,GUT_GENOME000010,MGBC000100,0.020884,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__O...
1,taxonomy,GUT_GENOME000035,MGBC000465,0.009662,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...
2,taxonomy,GUT_GENOME000049,MGBC109121,0.200162,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
3,taxonomy,GUT_GENOME000057,MGBC115383,0.008205,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...,d__Bacteria;p__Proteobacteria;c__Gammaproteoba...
4,taxonomy,GUT_GENOME000064,MGBC161554,0.625505,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...,d__Bacteria;p__Firmicutes;c__Bacilli;o__Lactob...


In [8]:
taxonomic_table = mgbc_clostest_taxa_table.query('method == "taxonomy" & reference_genome.str.contains("MGBC")')

In [9]:
# extract reference genus
taxonomic_table['reference_genus'] = taxonomic_table.reference_taxonomy.str.extract('(.*;g__[^;]+).*')

/tmp/ipykernel_39110/3870564890.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxonomic_table['reference_genus'] = taxonomic_table.reference_taxonomy.str.extract('(.*;g__[^;]+).*')


In [10]:
taxonomic_table['query_genus'] = taxonomic_table.query_taxonomy.str.extract('(.*;g__[^;]+).*')


/tmp/ipykernel_39110/2204288660.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxonomic_table['query_genus'] = taxonomic_table.query_taxonomy.str.extract('(.*;g__[^;]+).*')


In [11]:
# sanity check: we expect all the genus to be mapped to the same genus according to taxonomic distance
# some_genus = taxonomic_table['reference_genus'].iloc[0]
# taxonomic_table.query('reference_genus == @some_genus').query_genus.unique()
number_of_genus_mapped_per_reference_genus = taxonomic_table.groupby('reference_genus').apply(lambda x: len(x.query_genus.unique()))

In [12]:
number_of_genus_mapped_per_reference_genus.describe()

count    225.000000
mean       1.008889
std        0.094070
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        2.000000
dtype: float64

In [13]:
np.count_nonzero(number_of_genus_mapped_per_reference_genus == 2)

2

In [15]:
outlier_genus = number_of_genus_mapped_per_reference_genus[number_of_genus_mapped_per_reference_genus == 2].index
taxonomic_table.query('reference_genus in @outlier_genus[0]')[['reference_taxonomy', 'query_taxonomy', 'query_genus']]
# One of the species was not labeled to genus level, only to family level - which explain the difference (actually they are the same and we can ignore, we just need to verify we take the genus mapping and ignore the nuns.) 

,reference_taxonomy,query_taxonomy,query_genus
45836,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,NaN
45846,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...,d__Bacteria;p__Bacteroidota;c__Bacteroidia;o__...


In [16]:
taxonomic_table.query('reference_genus in @outlier_genus[1]')[['reference_taxonomy', 'query_taxonomy']]


,reference_taxonomy,query_taxonomy
45343,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45405,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45611,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45654,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45700,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45795,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45867,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45869,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45876,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...
45881,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...,d__Bacteria;p__Firmicutes_A;c__Clostridia;o__L...


We have 225 mice-genus in the MGBC table. 
We expected all the genus to be mapped to the same genus according to taxonomic distance, it's true mostly (for 223 genus)
But 2 genus have been mapped to 2 different genus based on their taxonomic distance:
Those genus are: 
'd__Bacteria;p__Bacteroidota;c__Bacteroidia;o__Bacteroidales;f__Bacteroidaceae;g__Paraprevotella' 
 'd__Bacteria;p__Firmicutes_A;c__Clostridia;o__Lachnospirales;f__Lachnospiraceae;g__Eubacterium_J'
 


Following consulting with Elhanan, we decided that this outlier is reasonable. And we will work with this table.
